In [ ]:
import speech_recognition as sr
import playsound 
import random
import webbrowser
import io
from gtts import gTTS
import pygame
import pyttsx3 
import pywhatkit
import datetime
import wikipedia
import pyjokes
import requests


In [ ]:
def speak(text):
    tts = gTTS(text=text, lang='en')
    byte_io = io.BytesIO()
    tts.write_to_fp(byte_io)
    byte_io.seek(0)

    pygame.mixer.init()
    sound = pygame.mixer.Sound(byte_io)
    sound.play()
    while pygame.mixer.get_busy():
        continue

def listen():
    r = sr.Recognizer()

    with sr.Microphone() as source:
        print("Listening...")
        r.pause_threshold = 1
        audio = r.listen(source)

    try: 
        query = r.recognize_google(audio, language='en-in')
        print(f"User said: {query}\n")
    except:
        print("Unable to Recognize your voice")
        return "None"

    return query.lower()

def respond(voice_data):
    print(voice_data)
    
    if 'what is your name' in voice_data:
        speak('My name is Alexis')
        
    elif 'what time is it' in voice_data: 
        time = datetime.datetime.now().strftime('%I:%M %p')
        speak(f"The current time is {time}")

    elif 'tell me a joke' in voice_data:
        joke = pyjokes.get_joke()  
        speak(joke)

    elif 'wikipedia' in voice_data:
        speak('Searching Wikipedia...')
        search_query = voice_data.replace('wikipedia', '')
        results = wikipedia.summary(search_query, sentences=2)
        speak("According to Wikipedia") 
        speak(results)
        
    elif 'weather' in voice_data:
        speak("Sure, which city's weather would you like to know?")
        city = listen()
        get_weather(city)

    elif 'news' in voice_data:
        speak("Sure, here are the latest headlines.")
        get_news()

    elif 'play music' in voice_data:
        speak("Sure, what song would you like to listen to?")
        song = listen()
        play_music(song)

    elif 'remind me to' in voice_data:
        speak("Sure, what would you like me to remind you?")
        reminder = listen()
        speak("After how many seconds should I remind you?")
        seconds = int(listen())
        set_reminder(reminder, seconds)

def get_weather(city):
    api_key = 'YOUR_OPENWEATHERMAP_API_KEY'
    url = f'http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&units=metric'
    response = requests.get(url)
    data = response.json()
    print(data)  # Print the response
    if data['cod'] == 200:
        weather_desc = data['weather'][0]['description']
        temp = data['main']['temp']
        speak(f"The weather in {city} is {weather_desc} with a temperature of {temp} degrees Celsius.")
    else:
        speak("Sorry, I couldn't retrieve the weather information for that location.")
        
def get_news():
    api_key = 'YOUR_NEWSAPI_KEY'  # Replace with your actual News API key
    url = f'https://newsapi.org/v2/top-headlines?country=us&apiKey={api_key}'
    response = requests.get(url)
    data = response.json()
    print(data)  # Print the response for debugging
    if 'articles' in data:
        articles = data['articles']
        for article in articles:
            speak(article['title'])
    else:
        speak("Sorry, I couldn't retrieve the news headlines. Please check your API key.")

def play_music(song):
    search_query = '+'.join(song.split())
    url = f"https://www.youtube.com/results?search_query={search_query}"
    webbrowser.open(url)

def set_reminder(reminder, seconds):
    speak(f"Reminder set for {reminder}.")
    time.sleep(seconds)
    speak(f"Time's up! Don't forget to {reminder}.")

if __name__ == '__main__':
    while True:
        voice_data = listen()
        respond(voice_data)